In [50]:
import pandas as pd
import re
from tqdm.notebook import tqdm

bungae_df = pd.read_csv('./new_bgzt_final_data.csv')
bungae_df['id'] = bungae_df['id'].astype(str)
bungae_df['cat_id'] = bungae_df['cat_id'].astype(str)
bungae_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203018 entries, 0 to 1203017
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1203018 non-null  object
 1   name            1203018 non-null  object
 2   image_url_list  1203018 non-null  object
 3   price           1203018 non-null  int64 
 4   info            1199641 non-null  object
 5   cat_id          1203018 non-null  object
 6   writed_at       1203018 non-null  object
 7   tag             1112801 non-null  object
 8   cat1            1203018 non-null  object
 9   cat2            1203018 non-null  object
 10  cat3            1133774 non-null  object
dtypes: int64(1), object(10)
memory usage: 101.0+ MB


In [51]:
# 새롭게 데이터 뽑기
from tqdm.notebook import tqdm
# -- 소분류 카테고리당 최대 700까지 줄이기 --
cat_id_list = list(bungae_df['cat_id'].unique())

new_df = pd.DataFrame()
for cat_id in tqdm(cat_id_list):
    tmp_df = bungae_df[bungae_df['cat_id'] == cat_id]
    if len(tmp_df) > 700: # 700개가 넘으면 700개만 가져오도록
        new_df = pd.concat([new_df, tmp_df.sample(700)], ignore_index=True)
    elif len(tmp_df) < 700: # 700개가 넘지 않으면 그대로 가져올 수 있도록
        new_df = pd.concat([new_df, tmp_df], ignore_index=True)

new_df = new_df.reset_index(drop=True)
print(len(new_df))

  0%|          | 0/157 [00:00<?, ?it/s]

109248


In [52]:
url_list = list()
for idx in tqdm(range(len(new_df))):
    tmp_url = new_df.loc[idx, 'image_url_list']
    url = re.sub('[\[\']', '', tmp_url.split(',')[0])
    
    url_list.append(url)

  0%|          | 0/109248 [00:00<?, ?it/s]

In [53]:
new_df['url'] = url

In [54]:
new_df = new_df[['id', 'name', 'cat_id', 'url']].copy()
new_df

,id,name,cat_id,url
0,207315989,루이비통패딩점퍼,310300200,https://media.bunjang.co.kr/product/109720327_...
1,213340343,플라스틱아일랜드 라쿤 후드 덕다운 롱패딩,310300200,https://media.bunjang.co.kr/product/109720327_...
2,179445605,**라쿤털 후드패딩,310300200,https://media.bunjang.co.kr/product/109720327_...
3,214139452,노이아고 NOI497 에센셜 다운 숏패딩,310300200,https://media.bunjang.co.kr/product/109720327_...
4,203871555,후아유 리버서블 숏패딩(플리스),310300200,https://media.bunjang.co.kr/product/109720327_...
...,...,...,...,...
109243,214388239,알라딘 셜록홈즈 금속 키링 열쇠고리 굿즈,400999,https://media.bunjang.co.kr/product/109720327_...
109244,81582461,새) 자주. 수납. 정리. 옷장 가방,400999,https://media.bunjang.co.kr/product/109720327_...
109245,219590266,귀마개,400999,https://media.bunjang.co.kr/product/109720327_...
109246,225809818,하회탈 이매탈 팝니다,400999,https://media.bunjang.co.kr/product/109720327_...


In [59]:
# new_df -> to csv
new_df.to_csv('./v2_newly_reduced_bungae_df.csv', index=False)

In [55]:
# cat_id, url, id 가져와서 리스트로 만들기
cat_url_prd_list = list()
for idx in tqdm(range(len(new_df))):
    cat_id = new_df.loc[idx, 'cat_id']
    url = new_df.loc[idx, 'url']
    prd_id = new_df.loc[idx, 'id']
    cat_url_prd_list.append('__'.join([cat_id, url, prd_id]))

  0%|          | 0/109248 [00:00<?, ?it/s]

In [57]:
# 텍스트 파일로 바꿔주기
with open('./bungae_cat_url_prd_list.txt', 'w+') as file: 
    file.write(','.join(cat_url_prd_list))

In [13]:
#  -- making folders --

In [58]:
# 1. folder 만들기 (maincat에 따라)

import os
import pandas as pd

def make_folders_for_image(df):
    cat_id_list = list(df['cat_id'].unique())
    
    for cat_id in cat_id_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'./bungae_images_v2/{cat_id}')
        
    return 

make_folders_for_image(new_df)

In [ ]:
# -- 2. 이미지 파일만 있는 데이터 로우 가져와서 새로운 데이터 프레임 만들어 주기--

In [60]:
import pandas as pd

df = pd.read_csv('./v2_newly_reduced_bungae_df')
df['id'] = df['id'].astype(str)
df['cat_id'] = df['cat_id'].astype(str)

# 데이터 수정 전 len
print(len(df))

FileNotFoundError: [Errno 2] No such file or directory: './v2_newly_reduced_bungae_df'

In [6]:
df.head()

,id,name,info,cat_id,tag,maincat_id,midcat_id
0,202546050,알토라쿤잠바,알토 라쿤잠바입니다 55사이즈네요 모자털 탈부착가능합니다,310300200,"알토,라쿤,잠바",310,310300
1,175146200,코스 오라리맛 푸퍼 다크브라운 40 판매합니다 (코스 다운 패디드 푸퍼),새상품 판매해요 택포,310300200,"코스푸퍼,코스,푸퍼",310,310300
2,213905890,[S] 커버낫 숏패딩,세탁완료! 깨끗하게 입었습니다 지퍼가 잘 됐다 안됐다 그럽니다;; 저는 똑딱이로 ...,310300200,"커버낫,숏,다운,오리털,패딩",310,310300
3,216998343,더 틸버리 구스패딩,55/한겨울 코트속에 이른봄은 외투로 활용도 높은 구스패딩입니다 컨디션최상~,310300200,더틸버리,310,310300
4,225515936,Sinoon 시눈 패딩 ELIN PATCHED PADDING,택 달려있어요 잠깐 외출할 때 몇번 입었구 하자 없습니다~!,310300200,"시눈,sinoon,패딩,프레클,스푸닝",310,310300


In [85]:
# 열리지 않는 이미지를 가지고 있는 행 제거하기
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# 파일이 열리는 row의 인덱스만 가져오기
idx_list = list()
for idx in tqdm(range(len(df))):
    prd_id = df.loc[idx, 'product_id']
    cat_id = df.loc[idx, 'cat_id']

    image_path = f'./bungae_images_v2/{cat_id}/{prd_id}.jpg'
    status=None
    try:
        plt.imread(image_path)
        idx_list.append(idx)
    except:
        pass

  0%|          | 0/98360 [00:00<?, ?it/s]